1. cleaning up compiled data from Nico 
- starting w/ one file for now = convnext.csv
- clean up columns w/ less than 10 data points and rows w/o data in "permanent" column
- "permanent" column = final diagnoses
- want to delete 128+ columns so Nico can add more image features 
(max # of columns readable by models we will use later = 500)
- want to separate the data via diagnoses to see what diagnoses works w/ what data

*preliminary data cleaning via chatgpt shows:
OG: 522 rows x 407 columns
cleaned: 510 rows x 241 columns
dropped columns: 166
dropped rows: 12
dx groups created: 220

*too many different dx
check top 10 most common dx
generate table of dx counts listing all 222 dx groups w/ row counts 
- easier to filter/sort by row count

In [4]:
import pandas as pd
import re
import os

In [ ]:
#cleaning up master data + creating dropped rows/columns file
# ✅ 1. Check current working directory (just to confirm)
print("Current working directory:", os.getcwd())

# ✅ 2. Helper function: Standardize ALL text entries
def standardize_text(value):
    if pd.isna(value):
        return value
    if isinstance(value, str):
        value = value.replace("\xa0", " ")          # replace non-breaking spaces
        value = re.sub(r"\s+", " ", value).strip()  # collapse multiple spaces & trim
        value = value.lower()                       # lowercase everything
        value = re.sub(r"^[-–]+", "", value).strip() # remove leading dashes or en-dashes
    return value

# ✅ 3. Cleaning & Standardizing Function
def clean_and_standardize_all(df, filename_prefix, save_path="./"):
    original_shape = df.shape

    # --- Standardize text in all object columns ---
    for col in df.columns:
        if df[col].dtype == "object":
            df[col] = df[col].apply(standardize_text)

   # --- Drop rows with no permanent diagnosis (NaN or empty) ---
    rows_before = df.shape[0]
    df_cleaned = df[df["permanent"].notna() & (df["permanent"].str.strip() != "")]

    # --- Track dropped rows ---
    rows_missing_dx = df[~df.index.isin(df_cleaned.index)].copy()
    rows_missing_dx["drop_reason"] = "No permanent diagnosis"

   # --- Drop columns with <10 non-null entries ---
    cols_to_drop = [col for col in df_cleaned.columns if df_cleaned[col].notna().sum() < 10]
    dropped_cols_info = pd.DataFrame({
        "column": cols_to_drop,
        "non_null_count": [df_cleaned[c].notna().sum() for c in cols_to_drop],
        "drop_reason": "Fewer than 10 non-null entries"
    })
    df_cleaned = df_cleaned.drop(columns=cols_to_drop)

    # ✅ Explicitly drop any unnamed/blank columns (just in case)
    unnamed_cols = [c for c in df_cleaned.columns if "Unnamed" in c or c.strip() == ""]
    if unnamed_cols:
        df_cleaned = df_cleaned.drop(columns=unnamed_cols)

    # --- ✅ Sort/group by permanent diagnosis ---
    df_cleaned = df_cleaned.sort_values(by="permanent").reset_index(drop=True)

    # --- Save Cleaned Master CSV ---
    cleaned_file = os.path.join(save_path, f"{filename_prefix}_cleaned_master.csv")
    df_cleaned.to_csv(cleaned_file, index=False)
    print(f"✅ Cleaned data saved (grouped by diagnosis): {cleaned_file}")

    # --- Save dropped info CSV ---
    dropped_cols_info["row_index"] = "N/A"
    rows_missing_dx["column"] = "N/A"
    rows_missing_dx["non_null_count"] = "N/A"
    dropped_info_combined = pd.concat([dropped_cols_info, rows_missing_dx], ignore_index=True)
    dropped_file = os.path.join(save_path, f"{filename_prefix}_dropped_info.csv")
    dropped_info_combined.to_csv(dropped_file, index=False)
    print(f"✅ Dropped info saved: {dropped_file}")

    return {
        "original_shape": original_shape,
        "cleaned_shape": df_cleaned.shape,
        "columns_dropped": len(cols_to_drop),
        "rows_dropped": rows_before - df_cleaned.shape[0]
    }

# ✅ 4. Example Run for ConvNeXt (change filename_prefix for others)
data_path = "/Users/joi263/Documents/MultimodalTabData/data/OG_data_csv/convnext_new.csv"
save_path = "/Users/joi263/Documents/MultimodalTabData/data/convnext_data"

df_convnext = pd.read_csv(data_path)
summary = clean_and_standardize_all(df_convnext, "convnext", save_path=save_path)
print(summary)


Current working directory: /Users/joi263/Documents/MultimodalTabData/data
✅ Cleaned data saved (grouped by diagnosis): /Users/joi263/Documents/MultimodalTabData/data/convnext_data/convnext_cleaned_master.csv
✅ Dropped info saved: /Users/joi263/Documents/MultimodalTabData/data/convnext_data/convnext_dropped_info.csv
{'original_shape': (522, 480), 'cleaned_shape': (510, 244), 'columns_dropped': 236, 'rows_dropped': 12}


In [24]:
#quick one liner fix for unnamed/blank columns - didn't want to resave master csv and manually delete again
file_path = "/Users/joi263/Documents/MultimodalTabData/data/convnext_data/convnext_cleaned_master.csv"

df = pd.read_csv(file_path)

# ✅ Drop any unnamed/blank columns and resave
df = df.loc[:, ~df.columns.str.contains("^Unnamed") & (df.columns.str.strip() != "")]
df.to_csv(file_path, index=False)

print(f"✅ Cleaned & resaved: {file_path}")
print(f"✅ Columns now: {df.shape[1]}")


✅ Cleaned & resaved: /Users/joi263/Documents/MultimodalTabData/data/convnext_data/convnext_cleaned_master.csv
✅ Columns now: 228


In [ ]:
#checking how many columns master csv now has - need to calculate how many were manually dropped
# Load your cleaned master CSV
df_cleaned = pd.read_csv("/Users/joi263/Documents/MultimodalTabData/data/convnext_data/convnext_cleaned_master.csv")

# Check the number of columns
num_columns = df_cleaned.shape[1]
print(f"✅ The cleaned master CSV has {num_columns} columns.")


✅ The cleaned master CSV has 228 columns.


2. cleaned convnext_new.csv (Jack's updated version w/ demographics) 
-went from 480 columns to 228 (252 columns dropped)
-manually deleted some columns using MasterDirectory for things I didn't think we'd need
(236 columns dropped via cleaning; 16 dropped manually)

3. now generate diagnosis count

4. also generate new csv that only has diagnoses w/ key words "glioma", "glioblastoma", and "meningioma"

In [ ]:
#diagnosis counts

def generate_diagnosis_counts(df, filename_prefix, save_path="./"):
    diagnosis_counts = (
        df.groupby("permanent")
        .size()
        .reset_index(name="row_count")
        .sort_values(by="row_count", ascending=False)
    )
    counts_file = os.path.join(save_path, f"{filename_prefix}_diagnosis_counts.csv")
    diagnosis_counts.to_csv(counts_file, index=False)
    print(f"✅ Diagnosis counts saved: {counts_file}")
    return diagnosis_counts

# ✅ Example Run (after cleaning)
df_cleaned = pd.read_csv("/Users/joi263/Documents/MultimodalTabData/data/convnext_data/convnext_cleaned_master.csv")
diagnosis_counts = generate_diagnosis_counts(df_cleaned, "convnext", save_path="/Users/joi263/Documents/MultimodalTabData/data/convnext_data")
diagnosis_counts.head(10)


✅ Diagnosis counts saved: /Users/joi263/Documents/MultimodalTabData/data/convnext_data/convnext_diagnosis_counts.csv


,permanent,row_count
83,"glioblastoma, cns who grade 4",71
82,glioblastoma who grade iv,40
177,pituitary adenoma,26
79,glioblastoma,20
129,meningioma,19
139,metastatic carcinoma,16
163,"oligodendroglioma, cns who grade 3",7
55,diffuse large b-cell lymphoma,7
124,lymphoma,7
172,pilocytic astrocytoma,6


In [26]:
#generate new file w/ key word diagnoses only - glioma, glioblastoma, meningioma

# Load the cleaned master CSV
master_path = "/Users/joi263/Documents/MultimodalTabData/data/convnext_data/convnext_cleaned_master.csv"
df_master = pd.read_csv(master_path)

# ✅ Filter rows where "permanent" contains glioma, glioblastoma, or meningioma (case-insensitive)
keywords = ["glioma", "glioblastoma", "meningioma"]
pattern = "|".join(keywords)  # creates "glioma|glioblastoma|meningioma"

df_filtered = df_master[df_master["permanent"].str.contains(pattern, case=False, na=False)]

# ✅ Save the filtered CSV
save_path = "/Users/joi263/Documents/MultimodalTabData/data/convnext_data/convnext_omas_only.csv"
df_filtered.to_csv(save_path, index=False)

print(f"✅ Filtered CSV saved: {save_path}")
print(f"✅ Shape: {df_filtered.shape} (rows, columns)")


✅ Filtered CSV saved: /Users/joi263/Documents/MultimodalTabData/data/convnext_data/convnext_omas_only.csv
✅ Shape: (273, 228) (rows, columns)
